# Part 1: Audio preprocessing (.wav to mfcc)

This notebook discusses the methods for processing **audio files** (speech in our case) to **MFCC** features. 

### Importing necessary variables and libraries along with GSC dataset

In [ ]:
## we load some variables from the config file
from config import *

## calculate the window length and hopsize in samples from window length and stride [ms]
lprime = l * int(fs / 1000)
hprime = lprime - s * int(fs / 1000)

In [ ]:
## we download the GSC dataset that we will use later
!python3 download_and_prepare_data.py

In [ ]:
## we load some of our own functions and libraries
from utility import load_all_wavs_in_dir

import matplotlib.pyplot as plt
import IPython.display as ipd
import numpy as np
import random 
import librosa

In [ ]:
## load background noise files to 'noise_data' for later use
noise_data = load_all_wavs_in_dir(direc=brn_directory, sr=16000)

### Visualization

We load a recording from the dataset and visualize it as a plot.

In [ ]:
y, rate = librosa.load(data_dir + "yes/0132a06d_nohash_0.wav", sr=fs)

In [ ]:
## We can plot it
plt.plot(y, alpha=1., color='blue')

In [ ]:
# Listen to it as well
ipd.Audio(y, rate=fs)

## Exercise 1

We want to build an audio pipeline for the **keyword spotter**. That means we need a function to transform the **.wav** files of our dataset into **spectrograms** or **mfcc** features. To do that, we will use the **librosa** (https://librosa.org/doc/latest/index.html) library.

1. First, derive a formula for the dimensions of the expected number of frames for a window length $l = 40 ms$, a stride $s = 20 ms$ and a duration of the audio signal of $T = 1s$. How many feature vectors will we get from our 1 second long audio signal?


2. Use librosa built in functions to firstly load a file from the dataset and secondly calculate the following:
- A short time Fourier Transformation,
- Calculate a mel spectrogram from it. Take the logarithm for better visualization. 
- Calculate mfccs from it. 

For each step, visualize the resulting spectrogram images using `plt.imshow()` 

3. Compare the results of the Fourier transform and mel spectrogram. What did the mel scaling achieve? Why was this useful?

## Hints:
- Use `librosa.stft`, `librosa.feature.melspectrogram`, `librosa.feature.mfcc`

## Solution

### E1


The function augments the input signal and maybe adds background noise, shifts audio etc.
The audio features are mfccs and are calculated with `librosa.feature.mfcc`, hopsize $h$ is passed as **parameter**. 

The hopsize $H$ in $ms$ is given by:
\begin{equation}
H  = L-S = 20\ ms
\end{equation}

The hopsize $h$ in samples, is:
\begin{equation}
h = H  \ f_s = 320
\end{equation}

The number of frames $N_f$ is: 
\begin{equation}
N_f = [\frac{N_s - s}{h}]   = 49
\end{equation}
where $N_s$ is the number of samples in the $1s$ audio signal. 

In [ ]:
(16000 - 320)/320 

### E2

In [ ]:
# Calculate the abs value of frequencies at time t
STFT_abs = np.abs(librosa.stft(y, n_fft=lprime, hop_length=hprime, win_length=lprime, center=False))

# Plot. Take the log of the values for better visibility
plt.imshow(np.log(STFT_abs**2), origin='lower', aspect='auto')
# plt.imshow(STFT_abs, origin='lower', aspect='auto')

plt.ylabel('frequency bin')
plt.xlabel('window number' )
plt.colorbar()
print('its hard to see the interesting, low frequency stuff!')

In [ ]:
## Rescale this to the mel scale and take log of the values for better visibility
MEL = librosa.feature.melspectrogram(S=STFT_abs, sr=fs, power=1, n_mels=100)

plt.imshow(np.log(MEL), origin='lower', aspect='auto')
plt.ylabel('log mel frequency bin')
plt.xlabel('window number' )
plt.colorbar()
print('now we see it better!')

The mel scaling emphasized the lower frequency parts of the spectrum. In the example we can nicely see the vowel $e$ from the word $yes$ as horizontal bands of fixed frequency. That helps for speech recognition!

In [ ]:
MFCC = librosa.feature.mfcc(S=np.log(MEL))
plt.imshow(MFCC, origin='lower', aspect='auto')
plt.ylabel('mfcc number')
plt.xlabel('window number' )
plt.colorbar()
print('finally convert the spectrogram to mfcc features')

# Part 2: Manipulating audio

Now, we know how to calculate mfcc features and have a feeling for what is going on when looking at spectrograms. In the next part, we will familiarize ourselves a bit more with audio signals by shifting them in time and adding background noise.

### Augmenting audio

In [ ]:
from utility import augment_audio

In [ ]:
## Load a sample audio file
y, rate = librosa.load(data_dir + "yes/0132a06d_nohash_0.wav", sr=None)

In [ ]:
## Function will add background noise to the audio signal:
y_augmented_noise = augment_audio(sig=y, # the signal
                            fs=fs, # sample frequency of the signal
                            padd_audio_to_samples=fs, # audio length is adjusted to this value
                            noise_data = noise_data, # use these noise data
                            background_frequency=1., # 1. means always add background noise
                            Ab=0.3, # with amplitude Ab
                            n_mfccs=0 # 0 --> get the signal directly
                           )

ipd.Audio(y_augmented_noise, rate=16000)

In [ ]:
## Also, calculate mfcc features
mfcc_augmented = augment_audio(silence=False, mode='training',
                  sig=y, fs=16000, padd_audio_to_samples=16000,
                  background_frequency=1., noise_data=noise_data, Ab=0.3,
                  l=40, s=20, n_mfccs=16)

mfcc_clean = augment_audio(silence=False, mode='training',
                  sig=y, fs=16000, padd_audio_to_samples=16000,
                  background_frequency=0., noise_data=noise_data, Ab=0.,
                  l=40, s=20, n_mfccs=16)

## Exercise

1. Implement a function called **shift** that shifts the audio signal by $ts$ seconds.

```
def shift(y, ts, fs):
    
    ## shift y
    if ts > 0.:
        
    elif ts < 0.:
    
    else:
        y_shifted = y
    
    return y_shifted
    
```

2. Take the signal $y$ from above and shift it. Compare the original and the shifted signals in a plot and with ipd.Audio. Does the method work?

3. Above we have already created two different mfcc versions of the recording **mfcc_augmented** and **mfcc_clean**. Plot them and compare. What did the addition of background noise do in the result?

## Hints
- Use `numpy.clip()`.
- For comparing the mfccs, use `fig,axs = plt.subplots(3,)` to create three plots and `axs[i].plot` and`axs[i].imshow()` to display the raw audio signal and the mfccs respectively.

### E1

In [ ]:
def shift(y, ts, fs):
    '''Randomly time shifts a signal y with sample frequency fs by up to +- ts'''
    
    ## Copy the signal
    y_shift = np.copy(y)
    
    shift_in_samples = int(np.abs(ts * fs))
    
    ## Perform the shift (different for ts>0, ts<0)
    if ts > 0:
        ## Add 0s to the left and take original signal length starting from sample 0:
        y_shift = np.pad(y_shift, (shift_in_samples, 0), mode='constant')[0:y_shift.shape[0]]
    elif ts < 0:
        ## Add 0s to the right and take original signal length starting from shift in samples:
        y_shift = np.pad(y_shift, (0, shift_in_samples), mode='constant')[
                  shift_in_samples:y_shift.shape[0] + shift_in_samples]

    return y_shift

### E2 


In [ ]:
## Plot shifted to see if it works
plt.plot(y, label="original")
y_shifted = shift(y=y, ts=0.2, fs=rate)
plt.plot(y_shifted, label="shifted")
plt.legend()

ipd.Audio(y, rate = rate)

In [ ]:
## Listen to shifted audio 
ipd.Audio(y_shifted, rate = rate)

### E3

In [ ]:
fig,axs = plt.subplots(3,)
fig.set_size_inches(18.5, 10.5)

axs[0].plot(y, color='blue')
axs[0].plot(y_augmented_noise, color='red', alpha=0.3)


i1 = axs[1].imshow(mfcc_clean.T, origin='lower', aspect='auto', label='clean')

i2 = axs[2].imshow(mfcc_augmented.T, origin='lower', aspect='auto', label='augmented')

fig.tight_layout()

### Comparison:
- Additive noise can be seen in the waveform.
- If noise comes from multiple (roughly equally distributed) frequency channels it can be seen in all mfcc coefficients.